In [5]:
import numpy as np
import pandas as pd
import cv2
import os
from keras import layers
from keras.layers import Input, Dense, Activation, Flatten, Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils, print_summary
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K
from keras import regularizers
from keras import optimizers
from keras.models import load_model
from sklearn.utils import shuffle

from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input

In [10]:

batch_size = 100

datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = datagen.flow_from_directory(
          'data/train',
          target_size=(50, 50),
          batch_size=batch_size,
          class_mode='categorical',  # this means our generator will only yield batches of data, no labels
          shuffle=True,
          color_mode='grayscale',
          classes=['1','2','3','4','5','6','7','8','9','10','11'])

validation_generator = datagen.flow_from_directory(
          'data/test',
          target_size=(50, 50),
          batch_size=batch_size,
          class_mode='categorical',  # this means our generator will only yield batches of data, no labels
          shuffle=True,
          color_mode='grayscale',
          classes=['1','2','3','4','5','6','7','8','9','10','11'])

Found 11000 images belonging to 11 classes.
Found 2200 images belonging to 11 classes.


In [7]:
model = VGG16()

In [11]:
#version1 is the best :)_
model1 = load_model('hand_emoji_v5.h5')
print(model1)

In [12]:
batch = 0 
x_generator = None
y_lable = None
for x,y in validation_generator:
    if batch == (2200/batch_size):
        break
    print("batch: ", batch)
    batch += 1
    if np.any(x_generator) == None:
        x_generator = model1.predict_on_batch(x)
        y_lable = y
        print(y)
    else:
        x_generator = np.append(x_generator,model1.predict_on_batch(x),axis=0)
        y_lable = np.append(y_lable,y,axis=0)
x_generator,y_lable = shuffle(x_generator,y_lable)
np.save('video_x_validate_VGG16.npy',x_generator)
np.save('video_y_validate_VGG16.npy',y_lable)


batch:  0
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
batch:  1
batch:  2
batch:  3
batch:  4
batch:  5
batch:  6
batch:  7
batch:  8
batch:  9
batch:  10
batch:  11
batch:  12
batch:  13
batch:  14
batch:  15
batch:  16
batch:  17
batch:  18
batch:  19
batch:  20
batch:  21


In [13]:
train_data = np.load('video_x_validate_VGG16.npy', mmap_mode='r')
train_labels = np.load('video_y_validate_VGG16.npy', mmap_mode='r')
train_data,train_labels = shuffle(train_data,train_labels)

In [14]:
train_data.shape

(2200, 12)

In [45]:
train_data.shape[1]

12

In [46]:
train_data = train_data.reshape(train_data.shape[0],
                train_data.shape[1] * train_data.shape[2],
                train_data.shape[3])

IndexError: tuple index out of range